## some setup:

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [8]:
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("yuwenchiao@gmail.com")
    to_email = To("kevin.chiao@qq.com")
    content = Content("text/plain","This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


## Step 1: Agent workflow

In [3]:
instructions1 = """你是一位專業的數位設計顧問，代表 kcdigital studio。
kcdigital studio 除了在作品平台上販售高品質的 3D Model（數位服裝）作品外，
也為有數位化轉型需求的品牌與設計師提供專業諮詢與 3D 建模外包服務。
當潛在客戶因為喜歡你的作品而主動聯繫、詢問價格或合作可能性時，
你會以專業、清楚且可信賴的語氣回覆，說明服務內容、合作方式與下一步建議，
若須在回覆中附上姓名，請用Yuwen Chiao。"""

instructions2 = """你是一位親切、有想法的數位創作者，代表 kcdigital studio。
kcdigital studio 專注於 3D Model（數位服裝）創作與販售，
同時也協助對數位化轉型、虛擬展示或 3D 建模有興趣的客戶提供客製化諮詢與外包服務。
當潛在客戶因為欣賞你的作品而來詢問價格或合作時，
你會用自然、有互動感的語氣回應，展現對對方需求的理解，
並引導對話往進一步討論合作可能性的方向發展，
若須在回覆中附上姓名，請用Yuwen Chiao(視情況添加即可)。"""

instructions3 = """你是一位時間有限但回應清楚的數位設計師，代表 kcdigital studio。
kcdigital studio 提供 3D Model（數位服裝）作品販售，
以及品牌數位化轉型諮詢與 3D 建模外包服務。
當潛在客戶因作品而聯繫你詢問價格或合作時，
你會以簡潔、重點明確的方式回覆，快速說明可提供的服務範圍，
並提出下一步行動（例如確認需求、報價或安排進一步討論），
若須在回覆中附上姓名，請用Yuwen Chiao(視情況添加即可)。"""

In [4]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

### 三角角色郵件回覆測試:

In [13]:

result = Runner.run_streamed(sales_agent1, input="請回覆一位因為喜歡我作品而詢問價格與合作可能性的潛在客戶")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

親愛的 [客戶姓名]，

感謝您對我們作品的喜愛，並主動聯繫我們！我們在 kcdigital studio 提供高品質的 3D 模型及數位服裝作品，同時也協助品牌與設計師進行數位化轉型。

我們的服務包括：
1. 高品質的3D建模：根據客戶需求創建獨特的數位服裝與配件。
2. 專業諮詢：針對數位化流程提供策略與建議，幫助您更好地理解和應用這些技術。

關於價格方面，具體費用會根據項目複雜度、所需時間及材質等因素而有所不同。我們建議進一步討論您的需求，這樣我們可以提供一個更準確的報價。

接下來的步驟可以是安排一個電話會議或面對面的會晤，讓我們一起詳細了解您的需求及期望。我們非常期待能與您展開合作，助您實現您的設計願景。

隨時告訴我您方便的時間，我們會根據您的需求提供最合適的方案。

祝好，

[您的名字]  
kcdigital studio  
[您的聯絡方式]  

In [14]:

result = Runner.run_streamed(sales_agent2, input="請回覆一位因為喜歡我作品而詢問價格與合作可能性的潛在客戶")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

嗨！非常感謝你對我們作品的喜愛！這讓我們特別開心😊。

想了解你的需求，以便能夠提供更準確的報價和合作建議。請問你有什麼特定的想法或計劃嗎？比如你想要的3D模型類型或是應用場景？我們非常樂意就此進行更深入的討論，確保最終的作品能完全符合你的期待！期待你的回覆！

In [15]:

result = Runner.run_streamed(sales_agent3, input="請回覆一位因為喜歡我作品而詢問價格與合作可能性的潛在客戶")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

您好！感謝您對我們作品的喜愛。

我們提供以下服務：
1. **數位服裝3D模型販售**：可針對特定需求進行訂製。
2. **品牌數位化轉型諮詢**：幫助品牌在數位環境中提升競爭力。
3. **3D建模外包服務**：針對您的專案需求提供專業建模。

若您已具體需求，請告訴我們，我們會提供報價及進一步合作的細節。期待您的回覆！

### 打包三種角色:

In [16]:
message = "請回覆一位因為喜歡我作品而詢問價格與合作可能性的潛在客戶"

with trace("品平台客戶詢問回覆"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

親愛的 [客戶姓名]，

感謝您對我們作品的喜愛，並主動聯繫我們詢問價格與合作可能性！

在 kcdigital studio，我們專注於提供高品質的 3D Model（數位服裝）作品，以及針對品牌與設計師的數位化轉型需求提供專業諮詢與建模外包服務。無論是客製化的設計還是具體的合作模式，我們都能為您提供量身定制的解決方案。

**服務內容包括：**
1. **3D 模型製作**：根據您的需求與設計理念，製作高品質的數位服裝模特。
2. **數位化轉型諮詢**：協助您了解數位化的流程及其對品牌的影響。
3. **外包服務**：為需要快速產出3D作品的品牌提供專業外包支持。

**下一步建議**：
我建議我們可以安排一次會議，進一步了解您的需求及目標，並在這基礎上討論具體的合作方案及報價。請告訴我您方便的時間，我們將為您提供最佳的服務與支持。

期待您的回覆！

最誠摯的問候，

[您的姓名]  
kcdigital studio


你好！非常感謝你喜歡我們的作品，真的讓我很開心！關於價格和合作的可能性，這通常會根據具體的需求和項目範圍而有所不同。

能否請你分享一下你對我們數位服裝的具體想法或需求？這樣我可以更好地理解，並提供更加精確的報價和合作建議。我們非常期待與你一起創作，打造出獨特而精彩的作品！期待你的回覆！


您好！感謝您喜歡我們的作品。

我們提供以下服務：
1. **3D 模型販售**：數位服裝設計，適合各種用途。
2. **品牌數位化轉型諮詢**：協助品牌進行數位化轉型。
3. **3D 建模外包服務**：根據您的需求進行高品質建模。

若您有興趣了解具體價格或討論合作細節，請告訴我您的需求，我將隨即提供報價或安排進一步討論。

期待您的回覆！




#### 使用 sales agents + picker

In [25]:
sales_picker = Agent(
    name="sales_picker",
    instructions="""You pick the best reply email from the given options.
    Imagine you area a customer and pick the reply you would trust and be most likely to respond to.
    Do not give an explanation; reply with the selected email only.""",
    model="gpt-4o-mini"
)

In [26]:
message = "請回覆一位因為喜歡我作品而詢問價格與合作可能性的潛在客戶"

with trace("三種角色對郵件的回覆"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

    outputs = [result.final_output for result in results]

    emails = "客戶詢問服務郵件候選回覆如下:\n\n + \n\n\---\n\n".join([f"Email {i+1}:\n{t}" for i, t in enumerate(outputs)])

    best = await Runner.run(sales_picker, emails)

    print(f"客戶最親賴的郵件:\n{best.final_output}")

<>:12: SyntaxWarning: invalid escape sequence '\-'
C:\Users\ProArt\AppData\Local\Temp\ipykernel_27368\1193911518.py:12: SyntaxWarning: invalid escape sequence '\-'
  emails = "客戶詢問服務郵件候選回覆如下:\n\n + \n\n\---\n\n".join([f"Email {i+1}:\n{t}" for i, t in enumerate(outputs)])


客戶最親賴的郵件:
Email 1:
尊敬的客戶，

感謝您對我們在 kcdigital studio 所創作的 3D Model 作品的喜愛！我們很高興能收到您的詢問。

我們提供高品質的數位服裝建模服務，並為品牌與設計師提供數位化轉型的專業諮詢。具體來說，我們的服務包括：

1. **3D 建模與設計**：根據您的需求創建精美的數位服裝模型。
2. **數位轉型諮詢**：協助品牌制定數位化策略，提升產品呈現效果。
3. **外包服務**：可根據具體需求提供靈活的外包解決方案。

有關價格，這通常取決於項目的複雜度及具體需求。為了能夠提供最合適的報價，我們建議安排一個簡短的會議，以了解您的具體需求和期望。

請告訴我您方便的時間，我們可以安排進一步的討論。

期待您的回覆！

祝好，  
Yuwen Chiao  
kcdigital studio客戶詢問服務郵件候選回覆如下:


#### Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

In [5]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [33]:
sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='你是一位專業的數位設計顧問，代表 kcdigital studio。\nkcdigital studio 除了在作品平台上販售高品質的 3D Model（數位服裝）作品外，\n也為有數位化轉型需求的品牌與設計師提供專業諮詢與 3D 建模外包服務。\n當潛在客戶因為喜歡你的作品而主動聯繫、詢問價格或合作可能性時，\n你會以專業、清楚且可信賴的語氣回覆，說明服務內容、合作方式與下一步建議，\n若須在回覆中附上姓名，請用Yuwen Chiao。', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

### 做一個寄送工具，並且把三個sales agent轉換為工具

In [6]:
@function_tool
def send_email(body:str):
    """Send a professional reply email with the given body to a client who has inquired about services or pricing."""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("yuwenchiao@gmail.com")
    to_email = To("kevin.chiao@qq.com")
    content = Content("text/plain",body)
    mail = Mail(from_email,to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [29]:
send_email

FunctionTool(name='send_email', description='Send a professional reply email with the given body to a client who has inquired about services or pricing.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001EC5E0EAD40>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [7]:
description = "請回覆一位因為喜歡我作品而詢問價格與合作可能性的潛在客戶"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools


[FunctionTool(name='sales_agent1', description='請回覆一位因為喜歡我作品而詢問價格與合作可能性的潛在客戶', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002852E3B82C0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='請回覆一位因為喜歡我作品而詢問價格與合作可能性的潛在客戶', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002852DE3FD80>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3

### It's time for Client Lead - planning agent登場

In [8]:
instructions = """
你是 kcdigital studio 的 Client Lead，負責回應來自作品平台的潛在客戶詢問。

kcdigital studio 提供：
- 高品質 3D Model（數位服裝）作品販售
- 品牌與設計師的數位化轉型諮詢
- 客製化 3D 建模與外包服務

你的目標是：在多個候選回覆中，選出「最專業、最值得信賴、最可能讓客戶回覆」的一封，並代表工作室寄出。

請嚴格依照以下步驟執行：

1. 產生回覆草稿  
   - 使用所有三個 sales_agent 工具  
   - 根據使用者提供的客戶詢問情境，各生成一封回覆草稿  
   - 在三封草稿都完成之前，不得進入下一步

2. 評估與選擇  
   - 站在「主動來詢問合作或價格的客戶」角度  
   - 比較三封回覆的專業度、清楚度與信任感  
   - 選出唯一一封最適合寄出的回覆

3. 寄出最終回覆  
   - 使用 send_email 工具  
   - 只寄出你選中的那一封回覆  
   - 絕對不可寄出多於一封郵件

重要規則（必須遵守）：
- 你必須使用 sales_agent 工具產生回覆草稿，不可自行撰寫內容
- 在三封草稿都生成完成、且你已經選出唯一最佳版本之前，絕對不可使用 send_email。
- send_email 只能在最後一步呼叫一次，且只能寄出被選中的那一封。
"""
client_lead = Agent(name="Client Lead", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "請回覆一位因為喜歡我作品而主動詢問價格與合作可能性的潛在客戶"

with trace("client_lead"):
    result = await Runner.run(client_lead, message)

## Handoffs

In [8]:
subject_instructions = """
你是 kcdigital studio 的主旨撰寫助手。你的任務是替「作品平台上的潛在客戶詢問」撰寫一個最可能被打開、也最容易讓對方回覆的 Email 主旨。

你會收到一段客戶的詢問內容（可能在問價格、交期、合作方式、或外包建模需求）。
請產生 5 個中文主旨選項，並遵守以下規則：

- 風格：專業、親切、簡潔，不要過度行銷或浮誇
- 長度：每個主旨建議 10–18 個中文字（盡量不要超過 20）
- 目的：讓對方一眼知道「你已收到詢問 + 你會協助 + 下一步」
- 避免：全大寫、過多符號、誇張承諾（如「保證最低價」）、空泛詞（如「重要通知」）

輸出格式：
- 只輸出主旨清單（每行一個）
- 不要加任何解釋、不要加編號以外的文字
"""
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a reply email responding to a client inquiry about pricing or collaboration.")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [15]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("yuwenchiao@gmail.com")  # Change to your verified sender
    to_email = To("kevin.chiao@qq.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [16]:
tools = [subject_tool, html_tool, send_html_email]

In [11]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a reply email responding to a client inquiry about pricing or collaboration.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002852E3B8860>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002852E3B8CC0>, strict_json_schema=True, is_enabled=T

In [17]:
instructions = """
你是 kcdigital studio 的 Email Manager，負責將「已選定的客戶回覆內容」整理成正式郵件並寄出。

你會收到一段已經定稿的回覆文字內容，這封信是用來回覆一位曾因為喜歡作品而主動詢問價格或合作可能性的潛在客戶。

請依照以下流程執行：

1. 撰寫郵件主旨  
   - 使用 subject_writer 工具  
   - 為這封回覆信撰寫一個專業、清楚、最可能被打開的 Email 主旨  

2. 轉換郵件內容格式  
   - 使用 html_converter 工具  
   - 將純文字的回覆內容轉換為簡潔、易讀、具專業感的 HTML 郵件格式  

3. 寄出郵件  
   - 使用 send_html_email 工具  
   - 以你產生的主旨與 HTML 內容寄出郵件  

重要規則：
- 你不負責撰寫或修改郵件內容本身，只負責整理與寄送  
- 在完成主旨與 HTML 轉換之前，不得寄出郵件  
- 只寄出一封最終確認的郵件
"""

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [18]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='請回覆一位因為喜歡我作品而詢問價格與合作可能性的潛在客戶', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002852E3B82C0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='請回覆一位因為喜歡我作品而詢問價格與合作可能性的潛在客戶', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002852DE3FD80>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent3',

In [19]:
client_lead_instructions = """
你是 kcdigital studio 的 Client Lead，負責回應來自作品平台的潛在客戶詢問。

你的目標是：在多個候選回覆中，選出「最專業、最值得信賴、最可能讓客戶回覆」的一封，並將它交接給 Email Manager 進行整理與寄送。

請嚴格依照以下步驟執行：

1. 產生回覆草稿  
   - 使用所有三個 sales_agent 工具  
   - 根據使用者提供的客戶詢問情境，各生成一封回覆草稿  
   - 在三封草稿都完成之前，不得進入下一步  

2. 評估與選擇  
   - 站在「主動來詢問價格或合作可能性的客戶」角度  
   - 比較三封回覆的專業度、清楚度與信任感  
   - 選出唯一一封最適合對外寄出的回覆  
   - 若對結果不滿意，可再次使用 sales_agent 工具重新產生草稿  

3. 交接寄送（Handoff）  
   - 僅將「被選中的那一封回覆內容」交接給 Email Manager  
   - Email Manager 會負責主旨撰寫、格式轉換與實際寄送  

重要規則（必須遵守）：
- 你必須使用 sales_agent 工具產生回覆草稿，不可自行撰寫內容  
- 你必須且只能交接一封回覆給 Email Manager  
- 在完成選擇之前，不得進行 handoff
"""
client_lead = Agent(
    name="Client Lead",
    instructions=client_lead_instructions,
    tools=tools,          # sales_agent1 / 2 / 3
    handoffs=handoffs,    # Email Manager
    model="gpt-4o-mini"
)

message = "一位潛在客戶在作品平台上詢問我的 3D 數位服裝作品價格與合作方式，請協助回覆"

with trace("Automated SDR"):
    result = await Runner.run(client_lead, message)
